link for outline
<a id='acquire_playlist_data_working'></a>

# Imports

In [4]:
import pandas as pd
import numpy as np
import requests

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src import api


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# What are we doing?

## Data pull stuff
- Get list of playlist urls for specific user to use in API endpoint
- Get songs off a specific playlist and store as a list
- Look up spotify data for songs in specific playlist and store them
- Figure out what raw data to use
- Download raw data for songs in specific playlist
- Operationalize raw data further

### Get user profile + playlist

#### Format POST requests (details in spotify_scratchpad.ipynb)

In [3]:
# CLIENT_ID = '1c2e33a8bebd41fbbb8a1ecf0e8c4273'
# CLIENT_SECRET = '4964098dcc7b41f99c4178e6403645c1'

# AUTH_URL = 'https://accounts.spotify.com/api/token'

# # POST

# auth_response = requests.post(AUTH_URL, {
#     'grant_type' : 'client_credentials', 
#     'client_id' : CLIENT_ID,
#     'client_secret' : CLIENT_SECRET,   
# })

# # convert response to json
# auth_response_data = auth_response.json()

# # save the access token
# access_token = auth_response_data['access_token']

#### Set correct headers for POST request

In [4]:
headers = {
    'Authorization' : 'Bearer {token}'.format(token=access_token)
    
}

#### Set base URL of all spotify API endpoints

In [5]:
BASE_URL = 'https://api.spotify.com/v1/'

#### Get profile

In [6]:
user_id = 'bothsidesdoit'

user_profile = requests.get(BASE_URL + 'users/' + user_id + '/playlists', headers=headers)

user_profile = user_profile.json()

#### Get playlist url endpoints for api

In [7]:
playlist_number = len(user_profile['items'])

playlist_urls = [
    user_profile['items'][number]['href']
    for number 
    in range(0, playlist_number)
]

In [7]:
playlist_urls

['https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6',
 'https://api.spotify.com/v1/playlists/2NcJyzjvUlvKQJsScpLOpu',
 'https://api.spotify.com/v1/playlists/1Zd7dmudNrNwpfShtK6ywf',
 'https://api.spotify.com/v1/playlists/1u6zqNx6dIAwvKHedLl0E2',
 'https://api.spotify.com/v1/playlists/5kXBwKZOEVT7PXV6rZuhnV',
 'https://api.spotify.com/v1/playlists/4fcUMfrQr7h3j6UpVndlKq']

#### Get specific playlist from above url

In [8]:
single_playlist_request = requests.get(
    playlist_urls[0],
    headers=headers
)

In [9]:
single_playlist_request = single_playlist_request.json()

#### Get list of song ids from the playlist request

In [10]:
single_playlist_ids = \
[
    single_playlist_request
        ['tracks']
        ['items']
        [track]
        ['track']
        ['id']
    for
        track
    in
        range(
            0, 
            len(single_playlist_request['tracks'])
        )
]

#### Get list of spotify features 

In [11]:
spotify_features_url = BASE_URL + 'audio-features/'

single_playlist_spotify_features_data = [
    requests.get(
        spotify_features_url + track_id,
        headers=headers
    )
    .json()
    for
        track_id
    in
        single_playlist_ids
]

#### Got the data!  Turning the above into a single function
#### input playlist url -> ouput data

In [9]:
def get_playlist_response(playlist_url):
    '''
    GET response for given playlist url from spotify api 
    
    input:
        playlist_url 
            - spotify API url for given playlist
            - string
        
    output:
        json of playlist_url RESPONSE from spotify api
    '''
    return requests.get(
        playlist_url,
        headers=headers
    ).json()
    

def parse_playlist_song_ids(playlist_url_response):
    '''
    parse playlist url RESPONSE for unique track IDs of songs in playlist 
    
    input:
        playlist_url_response
            - RESPONSE from spotify API for url for given playlist
            - json
        
    output:
        list of strings of unique track IDs 
    '''
    
    return [
    single_playlist_request
        ['tracks']
        ['items']
        [track]
        ['track']
        ['id']
    for
        track
    in
        range(
            0, 
            len(playlist_url_response['tracks'])
        )
    ]

def get_spotify_features_from_trackid(track_ids):
    '''
    GET response of spotify features for given list of track ids
    
    input:
        list of track ids
            - track id elements of list are strings
        
    output:
        df of spotify features for given track list 
    '''
    BASE_URL = 'https://api.spotify.com/v1/'
    
    spotify_features_url = BASE_URL + 'audio-features/'

    data = [
        requests.get(
            spotify_features_url + track_id,
            headers=headers
        )
        .json()
        for
            track_id
        in
            track_ids
    ]
    
    return pd.DataFrame(data)

def get_spotify_features(playlist_url):
    '''
    gather spotify features for each track in given playlist url
    
    input:
        playlist_url 
            - spotify API url for given playlist
            - string
        
    output:
        df of spotify features for that playlist
    '''
    
    playlist_response = get_playlist_response(
        playlist_url
    )
    
    playlist_track_ids = parse_playlist_song_ids(
        playlist_response
    )
    
    spotify_features = get_spotify_features_from_trackid(
        playlist_track_ids
    )
    
    return spotify_features

    
    

### Figure out what raw data to use

Goal is to be relatively quick 'n dirty but still provide info

In [2]:
playlist_response = api.get_playlist_response('https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6')

In [3]:
playlist_response

{'collaborative': False,
 'description': '',
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/1oqsGOPQF8TqVtiFdfqkd6'},
 'followers': {'href': None, 'total': 0},
 'href': 'https://api.spotify.com/v1/playlists/1oqsGOPQF8TqVtiFdfqkd6',
 'id': '1oqsGOPQF8TqVtiFdfqkd6',
 'images': [{'height': 640,
   'url': 'https://mosaic.scdn.co/640/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b273170683264052615fd667788dab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 640},
  {'height': 300,
   'url': 'https://mosaic.scdn.co/300/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b273170683264052615fd667788dab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 300},
  {'height': 60,
   'url': 'https://mosaic.scdn.co/60/ab67616d0000b2730c577c93959ad4f76aab306aab67616d0000b273170683264052615fd667788dab67616d0000b2735cb41ba5f1e01d3a932b2e24ab67616d0000b273faa00320d0021bd65b8613db',
   'width': 60}]

In [180]:
song_ids = api.parse_playlist_song_ids(playlist_response)

KeyError: 'tracks'

In [13]:
def get_track_raw(song_id):
    '''
    GET the low-level features of a single track through API call
    
    input:
        track_id - unique track ID in spotify api db
        str
        
    output:
        df of result of API call: low-level variables of single track
    '''
    
    data = requests.get(
    BASE_URL + 'audio-analysis/' + '6y0igZArWVi6Iz0rj35c1Y',
    headers=headers
    ) \
    .json()
    
    return = pd.DataFrame(data)

In [14]:
track_data = api.get_raw_data_track(song_ids[0])

In [176]:
track_dictionary = {
    'meta': ['status_code'],
    'track': [
        'duration', 'end_of_fade_in',
        'start_of_fade_out', 'window_seconds'
    ],
    'bars': 'all',
    'beats': 'all',
    'sections': 'all',
    'segments': 'all', 
    'tatums': 'all'
}

def unpack_selected_json(track_raw_json, track_raw_json_key, cols):
    '''
    unpack the nested dictionaries within specific key of track's low-level audio analysis data into df

    input:
        track_raw_json - JSON results from track audio analysis POST request
        json
        
        track_raw_json_key - specific key to unpack
        str
        
        cols - list of strings of columns to unpack eg ['key', 'tempo']
        
    output:
        df with one row (the track) and unpacked columns
    '''
    
    #individual data_groups are eg `section`, `beats` etc
    data_group = track_raw_json[track_raw_json_key]
    
    
    #syntax is eg
    # for every section in datagroup
    #    for every provided column
    #        create new dict entry section(int)_column:datagroup[section][column]
    unpacked_dict = {
        
        #create dict entry of eg `section10_tempo: '6.7'`
        '{}{}_{}'.format(track_raw_json_key, count, col):data_group[count][col] 
        
        #for every unit in the subgroup of data
        for count, _ in enumerate(data_group)
        
        #for every selected column in that unit
        for col in cols
    }
    
    #`pitch` and `timbre` in each `segment` are 12-dim vectors, need to unpack them
    #or else they won't fit onto one row of subsequent df
    
    key_list = list(unpacked_dict.keys())
    
    for key in key_list:
        update_dict = {}
        if 'pitch' in key:
            for count, item in enumerate(unpacked_dict[key]):
                update_dict.update({key+str(count):item})
            unpacked_dict.update(update_dict)
            del unpacked_dict[key]    
        if 'timbre' in key:
            for count, item in enumerate(unpacked_dict[key]):
                update_dict.update({key+str(count):item})
            unpacked_dict.update(update_dict)
            del unpacked_dict[key]  
    
    
    #index[0] to keep all the data on one row
    frame = pd.DataFrame(
        unpacked_dict, 
        index=[0]
    )
    
    return frame
    

def unpack_json(track_raw_json, columns_dictionary):
    '''
    unpack the nested dictionaries from a result of a POST request for a specific track's low-level audio analysis data

    input:
        track_raw_json - JSON results from track audio analysis POST request
        json
        
        columns_dictionary - keys are subgroups of data eg 'beats', items are either:
            'all' (str): flag to unpack all columns in subunit 
            list (list): list of strings of columns in subunit to unpack
        
    output:
        df with one row (the track) and unpacked columns
    '''
    
    frame_list = []
    
    for subgroup in columns_dictionary.keys():
        cols = columns_dictionary[subgroup]
        
        #subgroups with deeper lists unpack with unpack_selected_json function above
        if cols == 'all':
            
            cols = list(
                track_raw_json[subgroup][0].keys()
            )
        
            frame = unpack_selected_json(track_raw_json, subgroup, cols)
        
            frame_list.append(frame)            
        
        
        
        #subgroups with just key value pairs are unpacked here with similar structure
        #but w/o that additional nested list
        else:
            for col in cols:
                frame = pd.DataFrame({
                    '{}_{}'.format(subgroup, col):track_raw_json[subgroup][col]
                    },
                    index=[0]
                )

                frame_list.append(frame)

    return frame_list